In [1]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
import missingno
import os
import random
import pandas as pd
import shutil
from sklearn.base import BaseEstimator, TransformerMixin

In [2]:
# load data
data_path = "../../data/raw/"
train = pd.read_csv(os.path.join(data_path + "train_values.csv"))
labels = pd.read_csv(os.path.join(data_path + "train_labels.csv"))